## BNSTp ER $_{\alpha}$ Exploration

In [1]:
using Pkg
Pkg.activate("../..")
Pkg.status()

  Activating project at `~/Documents/Kaizen/code/theoretical-biology`


Status `~/Documents/Kaizen/code/theoretical-biology/Project.toml`
  [336ed68f] CSV v0.10.15
⌃ [13f3f980] CairoMakie v0.15.4
  [a93c6f00] DataFrames v1.7.0
  [0c46a032] DifferentialEquations v7.16.1
  [31c24e10] Distributions v0.25.120
  [38e38edf] GLM v1.9.0
⌃ [961ee093] ModelingToolkit v10.19.0
⌃ [91a5bcdd] Plots v1.40.16
⌃ [c3e4b0f8] Pluto v0.20.13
⌃ [7f904dfe] PlutoUI v0.7.68
  [6f49c342] RCall v0.14.9
  [f3b207a7] StatsPlots v0.15.7
  [0c5d862f] Symbolics v6.51.0
  [fdbf4ff8] XLSX v0.10.4
Info Packages marked with ⌃ have new versions available and may be upgradable.


## Load and explore data

### RNA-seq (MOESM4)

MOESM4 is a spreadsheet from Gegenhuber et al. 2022 that compares gene expression between estradiol and control samples

In [50]:
using XLSX, DataFrames

moesm4_path = "data/hormone-effects/Gegenhuber2022-RNAseq-adult.xlsx"
DEG_df = DataFrame(
    XLSX.gettable(
        XLSX.readxlsx(moesm4_path)["EB vs Veh"], 
        "A:G", 
        first_row=1
    )
)
rename!(DEG_df, [:gene, :baseMean, :log2FC, :lfcSE, :stat, :pvalue, :padj])
DEG_df.gene = uppercase.(DEG_df.gene)

topDEGs_df = filter(row -> row.padj < 0.05 && abs(row.log2FC) > 1.0, DEG_df)
topDEGs = Set(uppercase.(topDEGs_df.gene))
serialize("output/ERalphaDEGs", topDEGs)

println("Differential Expression Summary:")
println("----------------------------------------------------------------")
println("Total # of DEGs: ", nrow(topDEGs_df))
println("Significant genes (padj < 0.05, |log2FC| > 1): ", nrow(topDEGs_df))
println("Upregulated: ", sum(topDEGs_df.log2FC .> 1))
println("Downregulated: ", sum(topDEGs_df.log2FC .< -1))
println("DEGs: $([gene for gene in topDEGs])")

Differential Expression Summary:
----------------------------------------------------------------
Total # of DEGs: 22
Significant genes (padj < 0.05, |log2FC| > 1): 22
Upregulated: 11
Downregulated: 11
DEGs: ["MOB3C", "DENR", "RCN1", "2310030G06RIK", "MARVELD3", "PALB2", "ITGA7", "GLIS2", "L3MBTL3", "PHLDA1", "NXN", "RGS16", "PTCHD2", "GPR160", "TACO1", "RFX2", "BRINP2", "FANCM", "SUSD5", "NGFR", "2310033P09RIK", "ZAN"]


### CUT&RUN (MOESM3)

MOESM3 also from Gegenhuber et al. 2022 details where exactly $\text{ER}{\alpha^{+}}$ binds thereby revealing candidate regualtors (upstream inputs)

In [25]:
using XLSX, DataFrames

moesm3_path = "data/hormone-effects/Gegenhuber2022-ERalpha-CUTRUN-adult.xlsx"
moesm3_xf = XLSX.readxlsx(moesm3_path)

# moesm3_sheets = XLSX.sheetnames(moesm3_xf)
# for (i, sheet) in enumerate(moesm3_sheets)
#     println("Sheet $(i): ", sheet)
# end

moesm3_sheet = "E2 peaks"
CUTRUN_df = DataFrame(
    XLSX.gettable(
        moesm3_xf[moesm3_sheet],
    )
)

rename!(CUTRUN_df, Dict("Gene annotation" => :gene))
CUTRUN_df.gene .= uppercase.(CUTRUN_df.gene)

# find DEGs directly responsive to ERalpha
CUTRUN_topDEGs = filter(
    row -> row.gene in topDEGs, 
    CUTRUN_df
)
unique(CUTRUN_topDEGs.gene)

2-element Vector{String}:
 "BRINP2"
 "RCN1"

### Differentially Expressed TFs

We find 22 strong DEGs from RNA-seq → only 2 of them (Brinp2, Rcn1) have direct ERα binding in CUT&RUN.

Those 2 are effectors (structural/signaling genes), not transcription factors.

In a GRN, effectors don’t usually regulate other genes — they’re the end of the chain. The other 20 DEGs could be indirectly regulated by ERα through intermediate transcription factors.

#### Define TF universe with differential expression values


In [51]:
using CSV

mouseTFs = Set(
    uppercase.(strip.(
        CSV.read(
            "data/reference/mm_mgi_tfs.txt", DataFrame; 
            header=false
        )[:,1]
    ))
)

RNAseqgenes = Set(DEG_df.gene) # all genes for which we have RNA-seq data

bound_DETFs = filter(
    row -> (row.gene in mouseTFs) && (row.gene in RNAseqgenes) && row.FDR < 0.05,
    CUTRUN_df
)

sort!(bound_DETFs, :FDR)

Row,chromosome,start,end,size,strand,Conc,Conc_E2,Conc_Veh,Fold,p-value,FDR,gene
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,String
1,chr12,12872907,12873107,201,*,3.54,4.38,1.32,3.07,4.62e-8,9.97e-6,MYCN
2,chr17,86570591,86570791,201,*,4.19,5.04,1.8,3.24,1.13e-7,1.95e-5,EPAS1
3,chr17,88329018,88329296,279,*,3.66,4.55,0.85,3.71,2.81e-7,3.71e-5,FOXN2
4,chr9,8837711,8837911,201,*,4.15,4.99,1.86,3.13,3.53e-7,4.47e-5,PGR
5,chr9,8937798,8937998,201,*,4.12,4.9,2.3,2.6,4.11e-7,5.07e-5,PGR
6,chr7,55043841,55044041,201,*,4.47,5.34,1.87,3.48,4.77e-7,5.58e-5,LUZP2
7,chr5,27652812,27653104,293,*,3.95,4.83,1.38,3.45,1.16e-6,0.00011,PAXIP1
8,chr1,168243842,168244042,201,*,3.75,4.62,1.19,3.44,6.47e-6,0.000437,PBX1
9,chr15,77231817,77232017,201,*,2.82,3.64,0.71,2.93,7.07e-6,0.000468,RBFOX2


### Construct Layer 1 (ER $_{\alpha} \to$ DETFs)

In [38]:
DEG_sub = select(DEG_df, [:gene, :baseMean, :log2FC, :padj])

layer1_join = innerjoin(bound_DETFs, DEG_sub, on = :gene)

layer1_edges = DataFrame(
    source   = fill("ERALPHA", nrow(layer1_join)),
    target   = layer1_join.gene,
    sign     = ifelse.(layer1_join.log2FC .> 0, "Activator", "Repressor"),
    weight   = .-log10.(layer1_join.FDR .+ eps()),   # evidence strength
    evidence = fill("CUT&RUN_FDR<0.05", nrow(layer1_join))
)

Row,source,target,sign,weight,evidence
,String,String,String,Float64,String
1,ERALPHA,PGR,Activator,4.34969,CUT&RUN_FDR<0.05
2,ERALPHA,PGR,Activator,4.29499,CUT&RUN_FDR<0.05
3,ERALPHA,PGR,Activator,3.02319,CUT&RUN_FDR<0.05
4,ERALPHA,PGR,Activator,2.87615,CUT&RUN_FDR<0.05
5,ERALPHA,NR2F1,Repressor,2.10347,CUT&RUN_FDR<0.05
6,ERALPHA,ODC1,Activator,1.52288,CUT&RUN_FDR<0.05
7,ERALPHA,SOX18,Activator,1.46852,CUT&RUN_FDR<0.05
8,ERALPHA,RREB1,Repressor,1.40561,CUT&RUN_FDR<0.05
9,ERALPHA,ZXDC,Activator,2.33724,CUT&RUN_FDR<0.05


### Start Nodes Database

Each gene (node) will eventually need $\alpha, \beta, \text{K}, n$ derived from empirical data such as the RNA-seq information

In [48]:
nodes_db = DataFrame(
    gene    = layer1_join.gene,
    baseMean = layer1_join.baseMean,   # proxy steady state
    log2FC  = layer1_join.log2FC,
    padj    = layer1_join.padj,
    class   = fill("TF", nrow(layer1_join))   # mark as TFs (can add other classes later)
)


Row,gene,baseMean,log2FC,padj,class
,String,Any,Any,Any,String
1,PGR,1419.67,0.723679,0.040434,TF
2,PGR,1419.67,0.723679,0.040434,TF
3,PGR,1419.67,0.723679,0.040434,TF
4,PGR,1419.67,0.723679,0.040434,TF
5,NR2F1,500.926,-0.807142,0.0732341,TF
6,ODC1,724.235,0.714085,0.17633,TF
7,SOX18,279.255,1.43359,0.258748,TF
8,RREB1,1321.48,-0.509154,0.47496,TF
9,ZXDC,806.159,0.633613,0.542747,TF


### Using DOROTHEA to determine targets of DETFs

In [59]:
using RCall

R"""
library(dorothea)
data(dorothea_mm)
df <- subset(dorothea_mm, confidence %in% c("A","B","C", "D", "E"))
"""
dorothea = rcopy(R"df")

dorothea.tf = uppercase.(dorothea.tf)
dorothea.target = uppercase.(dorothea.target)
first(dorothea, 7)

Row,tf,confidence,target,mor
,String,String,String,Float64
1,4932411N23RIK,E,SMAD4,1.0
2,4932411N23RIK,E,0610030E20RIK,1.0
3,4932411N23RIK,E,1700017N19RIK,1.0
4,4932411N23RIK,E,4931428F04RIK,1.0
5,4932411N23RIK,E,4932438A13RIK,1.0
6,4932411N23RIK,E,4933424G06RIK,1.0
7,4932411N23RIK,E,4933427D14RIK,1.0


In [61]:
deg_names = Set(topDEGs)
dor_targets = Set(dorothea.target)
println("Overlap DEGs ∩ dorothea targets = ", intersect(deg_names, dor_targets))


# TF regulators of your DEGs according to dorothea
deg_regulators = unique(filter(row -> row.target in topDEGs, dorothea).tf)

# which of these are in your Layer1 TF set?
deg_regulators_in_layer1 = intersect(Set(deg_regulators), Set(layer1_edges.target))
deg_regulators_missing   = setdiff(Set(deg_regulators), Set(layer1_edges.target))

println("Regulators of DEGs that ARE in Layer1: ", deg_regulators_in_layer1)
println("Regulators of DEGs that are NOT in Layer1: ", deg_regulators_missing)


Overlap DEGs ∩ dorothea targets = Set(["MOB3C", "DENR", "RCN1", "MARVELD3", "PALB2", "ITGA7", "GLIS2", "L3MBTL3", "PHLDA1", "NXN", "RGS16", "GPR160", "TACO1", "RFX2", "BRINP2", "FANCM", "SUSD5", "NGFR", "2310033P09RIK", "ZAN"])
Regulators of DEGs that ARE in Layer1: Set(["KLF12", "ZFHX3", "FOXN2", "EGR3", "ZBTB46", "PBX1"])
Regulators of DEGs that are NOT in Layer1: Set(["CBX2", "MXD1", "ZFP661", "FOXD3", "ZFP719", "ZFP64", "TBX6", "ZFP729A", "ZFP667", "NPAS4", "GLIS3", "MEOX2", "ZFP653", "ADNP", "FOXS1", "CEBPZ", "CREBL2", "MEIS3", "GCM1", "PRDM5", "NR2C1", "CIC", "NR3C2", "ZBTB24", "FOXF1", "HIVEP2", "ZFP212", "ZFP78", "EBF4", "TBX1", "TSHZ3", "ZFP3", "ZBTB17", "HOXA13", "ZFP664", "TFAP2B", "KLF8", "ZFP423", "ATF5", "ETV3", "IRF6", "ZFP738", "ZFP956", "THAP3", "EN2", "ALX3", "BARX1", "MSX1", "ZHX2", "GSX2", "NRL", "ZKSCAN6", "HOXB3", "ZBTB7C", "BNC2", "INSM1", "OVOL1", "ALX1", "ZBTB14", "HESX1", "GBX2", "SCX", "SETDB1", "PRDM4", "ZBTB7B", "ARNTL2", "ZBTB6", "CSRNP2", "ELK3", "HEY2", 

### Construct Layer 2 Edges

In [ ]:
layer1_TFs = unique(layer1_edges.target)

layer2_edges = filter(
    row -> (row.tf in layer1_TFs) && (row.target in RNAseqgenes),
    dorothea
)

confidence_map = Dict("A" => 3, "B" => 2, "C" => 1)

layer2_edges.source   = layer2_edges.tf
layer2_edges.weight   = [confidence_map[c] for c in layer2_edges.confidence]
layer2_edges.evidence = "DoRothEA_conf" .* layer2_edges.confidence
layer2_edges.sign     = ifelse.(layer2_edges.mor .== 1.0, "Activator", ifelse.(layer2_edges.mor .== -1.0, "Repressor", "Unknown"))

layer2_edges = select(layer2_edges,
    [:source, :target, :sign, :weight, :evidence]
)


Row,source,target,sign,weight,evidence
,String,String,String,Int64,String
1,AR,1700025G04RIK,Activator,3,DoRothEA_confA
2,AR,ABCE1,Activator,3,DoRothEA_confA
3,AR,AKT1,Activator,3,DoRothEA_confA
4,AR,ANAPC10,Activator,3,DoRothEA_confA
5,AR,AP2M1,Activator,3,DoRothEA_confA
6,AR,APPBP2,Activator,3,DoRothEA_confA
7,AR,ATF6,Activator,3,DoRothEA_confA
8,AR,ATM,Activator,3,DoRothEA_confA
9,AR,ATP2B4,Activator,3,DoRothEA_confA


### Extend Nodes Database

In [49]:
new_nodes = innerjoin(DataFrame(gene = unique(layer2_edges.target)), DEG_sub, on=:gene)

new_nodes.class .= "target"   # these are downstream, not TFs
rename!(new_nodes, Dict(:baseMean => :baseMean))

nodes_db = vcat(nodes_db, new_nodes)


Row,gene,baseMean,log2FC,padj,class
,String,Any,Any,Any,String
1,PGR,1419.67,0.723679,0.040434,TF
2,PGR,1419.67,0.723679,0.040434,TF
3,PGR,1419.67,0.723679,0.040434,TF
4,PGR,1419.67,0.723679,0.040434,TF
5,NR2F1,500.926,-0.807142,0.0732341,TF
6,ODC1,724.235,0.714085,0.17633,TF
7,SOX18,279.255,1.43359,0.258748,TF
8,RREB1,1321.48,-0.509154,0.47496,TF
9,ZXDC,806.159,0.633613,0.542747,TF


In [52]:
GRN_edges_full = vcat(layer1_edges, layer2_edges)
CSV.write("output/GRN_edges_full.csv", GRN_edges_full)

"output/GRN_edges_full.csv"

## Induce ER $_{\alpha} \to \dots \to$ DEG subgraph

In [54]:
covered_DEGs = intersect(Set(layer2_edges.target), topDEGs)
missing_DEGs = setdiff(topDEGs, covered_DEGs)
println(("covered", covered_DEGs))
println(("missing", missing_DEGs))


("covered", Set{String}())
("missing", Set(["MOB3C", "DENR", "RCN1", "2310030G06RIK", "MARVELD3", "PALB2", "ITGA7", "GLIS2", "L3MBTL3", "PHLDA1", "NXN", "RGS16", "PTCHD2", "GPR160", "TACO1", "RFX2", "BRINP2", "FANCM", "SUSD5", "NGFR", "2310033P09RIK", "ZAN"]))


In [53]:
function induce_subgraph(edges::DataFrame, DEGs::Vector{String}; driver="ERALPHA")
    # Backward closure from DEGs
    keep = Set(DEGs)
    updated = true
    while updated
        updated = false
        for r in eachrow(edges)
            if (r.target in keep) && !(r.source in keep)
                push!(keep, r.source)
                updated = true
            end
        end
    end
    edges_back = filter(r -> r.target in keep, edges)

    # Forward reachability from ERALPHA
    reach = Set([driver])
    updated = true
    while updated
        updated = false
        for r in eachrow(edges_back)
            if (r.source in reach) && !(r.target in reach)
                push!(reach, r.target)
                updated = true
            end
        end
    end
    edges_sub = filter(r -> (r.source in reach) && (r.target in reach), edges_back)

    # Report unreachable DEGs
    unreachable = setdiff(Set(DEGs), intersect(Set(DEGs), reach))
    return edges_sub, collect(unreachable)
end

edges_sub, unreachable_DEGs = induce_subgraph(GRN_edges_full, collect(topDEGs))
println(("Edges in subgraph", nrow(edges_sub)))
println(("Unreachable DEGs", unreachable_DEGs))


("Edges in subgraph", 0)
("Unreachable DEGs", ["MOB3C", "DENR", "RCN1", "2310030G06RIK", "MARVELD3", "PALB2", "ITGA7", "GLIS2", "L3MBTL3", "PHLDA1", "NXN", "RGS16", "PTCHD2", "GPR160", "TACO1", "RFX2", "BRINP2", "FANCM", "SUSD5", "NGFR", "2310033P09RIK", "ZAN"])
